In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vendas-modificado/vendas_modificado.csv


In [2]:
limp = pd.read_csv('/kaggle/input/vendas-modificado/vendas_modificado.csv')

In [3]:
limp.head()

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,"R$ 16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"R$ 3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,"R$ 1,63",9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,Carvão,"R$ 8,74",4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,"R$ 9,48",2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações


In [4]:
print(limp.dtypes)

id_da_compra      int64
data             object
hora             object
cliente          object
produto          object
valor            object
quantidade        int64
total           float64
status           object
cidade           object
estado           object
pais             object
cep              object
frete           float64
pagamento        object
vendedor         object
marca            object
dtype: object


Verificação e Padronização de Tipos e Formatos

Formatando a data e a hora

In [5]:
limp['data'] = pd.to_datetime(limp['data'], errors='coerce').dt.strftime('%Y-%m-%d')

In [6]:
limp['hora'] = pd.to_datetime(limp['hora'], format='%H:%M:%S', errors='coerce').dt.strftime('%H:%M:%S')

In [7]:
print(limp[['data', 'hora']].head())

         data      hora
0  2021-03-20  23:35:51
1  2020-10-30  09:00:53
2  2021-06-09  15:30:28
3  2022-06-04  08:41:23
4  2019-05-04  13:38:45


Convertendo as colunas para formato numérico

In [8]:
colunas_numericas = ['quantidade', 'total', 'frete']
for coluna in colunas_numericas:
    limp[coluna] = pd.to_numeric(limp[coluna], errors='coerce')

Verificando os tipos de dados após converte-los

In [9]:
print(limp.dtypes)

id_da_compra      int64
data             object
hora             object
cliente          object
produto          object
valor            object
quantidade        int64
total           float64
status           object
cidade           object
estado           object
pais             object
cep              object
frete           float64
pagamento        object
vendedor         object
marca            object
dtype: object


Tratando as Inconsistências

corrijindo a coluna de produto pois pode ajudar com os valores ausentes

In [10]:
print(limp[['produto']].head())

            produto
0  Queijo Mussarela
1   Molho de Tomate
2      Água Mineral
3            Carvão
4              Café


Importando a biblioteca (re) pois ela ajuda a trabalhar com expressões regulares

In [11]:
import re

Removendo caracteres especiais da coluna 'produto'

In [12]:
limp['produto'] = limp['produto'].astype(str).apply(
    lambda x: re.sub(r'[^A-Za-z0-9À-ÿ\s]', '', x)
)

In [14]:
from fuzzywuzzy import process

produtos_unicos = limp['produto'].unique()
for prod in produtos_unicos:
    print(process.extract(prod, produtos_unicos, limit=3))

[('Queijo Mussarela', 100), ('Qbeijo Mussarela', 94), ('Queijo Mussarelz', 94)]
[('Molho de Tomate', 100), ('Mopho de Tomate', 93), ('Molmo de Tomate', 93)]
[('Água Mineral', 100), ('Água Mineras', 91), ('Água Mineual', 91)]
[('Carvão', 100), ('Açúcar', 67), ('Macarrão', 67)]
[('Café', 100), ('Caft', 86), ('Caff', 86)]
[('Feijão', 100), ('ieijão', 80), ('Queijo Mussarela', 72)]
[('Condicionador', 100), ('Condicioiador', 92), ('Condibionador', 92)]
[('Sabonete', 100), ('zabonete', 88), ('Sabonepe', 88)]
[('Açúcar', 100), ('Macarrão', 90), ('Açúcaz', 75)]
[('Manteiga', 100), ('Manteigt', 88), ('Molho de Tomate', 60)]
[('Papel Higiênico', 100), ('Papel Toalha', 56), ('Papel qoalha', 56)]
[('Óleo de Soja', 100), ('Pão de Forma', 64), ('Molho de Tomate', 62)]
[('Vinho', 100), ('Papel Higiênico', 54), ('Farinha de Trigo', 54)]
[('Macarrão', 100), ('Açúcar', 90), ('Macawrão', 86)]
[('Shampoo', 100), ('Sal', 60), ('Sabão em Pó', 50)]
[('Presunto', 100), ('Presuntd', 88), ('Papel Toalha', 45)]


corrigindo produtos

In [17]:
correcao_produto = {
    "Qbeijo Mussarela": "Queijo Mussarela",
    "Queijo Mussarelz": "Queijo Mussarela",
    "Mopho de Tomate": "Molho de Tomate",
    "Água Mineras": "Água Mineral",
    "Água Mineual": "Água Mineral",
    "Condicioiador": "Condicionador",
    "Condibionador": "Condicionador",
    "zabonete": "Sabonete",
    "Sabonepe": "Sabonete",
    "Macawrão": "Macarrão",
    "Manteigt": "Manteiga",
    "Papel qoalha": "Papel Toalha",
    "Papel Twalha": "Papel Toalha",
    "Presuntd": "Presunto",
    "Amaciayte": "Amaciante",
    "Desinfetanue": "Desinfetante",
    "Desinfekante": "Desinfetante",
    "Refrigkrante": "Refrigerante",
    "Arroc": "Arroz",
    "Deqergente": "Detergente",
    "Deterwente": "Detergente",
    "Cafc": "Café",
    "Caft": "Café"
}
limp['produto'] = limp['produto'].replace(correcao_produto)

In [18]:
print(limp[['produto']].head())

            produto
0  Queijo Mussarela
1   Molho de Tomate
2      Água Mineral
3            Carvão
4              Café


Lidando com valores Ausentes

In [19]:
print(limp.isnull().sum())

id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor              0
quantidade         0
total           3685
status             0
cidade             0
estado             0
pais               0
cep                0
frete           7371
pagamento          0
vendedor        3680
marca              0
dtype: int64


valores Faltantes e a Escolha do Tratamento Deoendendo da Coluna
Total: Recalcular se Valor,Quantidade e Frete se Esteverem Corretos
Frete : Media
Vendedor : Remoçao ou Aplicar a Palavra Desconhecido

Tratando o Valor

In [20]:
limp['valor'] = limp['valor'].str.replace(r'R\$', '', regex=True).str.strip()

In [21]:
limp.head(3)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,"16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,"1,63",9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba


Organizando o Estado Correto

In [24]:
estado_correto = {
    'São Paulo': 'SP',
    'MTSa': 'MT',
    'PSC': 'SC'
}
limp['estado'] = limp['estado'].replace(estado_correto)

Tratando do Frete

In [ ]:
limp['Frete_Check1'] = limp.groupby(['cidade', 'estado'])['frete'].transform('median')

In [25]:
limp.head(3)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,"16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,"1,63",9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba


In [27]:
limp['Frete_Check2'] = limp.frete.fillna(limp.groupby (['cidade', 'estado'])['frete'].transform('median'))

In [28]:
limp = limp[(limp['frete'] >= 0) & (limp['total'] >= 0)]

/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)


In [29]:
limp.head(3)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,Frete_Check2
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,"16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre,20.0
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini,0.0
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,"1,63",9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba,22.0


In [ ]:
limp['frete'] =   limp.frete.fillna(limp.groupby (['cidade', 'estado'])['frete'].transform('median'))


In [ ]:
limp = limp.drop(columns=['Frete_Check1', 'Frete_Check2'])

In [30]:
limp.head(3)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,Frete_Check2
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,"16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre,20.0
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini,0.0
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,"1,63",9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba,22.0


In [31]:
print(limp.isnull().sum())

id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor              0
quantidade         0
total              0
status             0
cidade             0
estado             0
pais               0
cep                0
frete              0
pagamento          0
vendedor        3518
marca              0
Frete_Check2       0
dtype: int64


VERIFICAÇÃO DO VALOR TOTAL

Pegando o Primeiro Número com Vírgula Decimal

In [32]:
def corrigir_valor(valor):
    match = re.search(r'\d{1,3}(,\d{2})?', str(valor))  
    if match:
        return match.group().replace(',', '.')
    return np.nan

limp['valor'] = limp['valor'].apply(corrigir_valor).astype(float)

Calculando o total correto

In [33]:
limp['total_calculado'] = limp['valor'] * limp['quantidade'] + limp['frete']

In [34]:
limp[limp.total.isnull()].head(6)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,Frete_Check2,total_calculado


In [35]:
limp['total'] = np.where(
    np.abs(limp['total'] - limp['total_calculado']) < 0.01,
    limp['total'],
    limp['total_calculado']
)


In [36]:
limp.drop(columns=['total_calculado'], inplace=True, errors='ignore')


In [37]:
limp.head(6)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,Frete_Check2
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,16.87,13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre,20.0
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,3.25,3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini,0.0
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,1.63,9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba,22.0
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,Carvão,8.74,4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás,20.0
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,9.48,2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações,0.0
5,38623,2018-02-19,17:32:01,LUCAS ANTÔNIO DE SOUZA NETO,Café,10.16,1,20.16,Entregue com Sucesso,Resende Costa,MG,Brasil,36340-000,10.0,Transferência Bancária,VICTOR GONÇALVES DONADONI,3 Corações,10.0


In [38]:
print(limp.isnull().sum())

id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor              0
quantidade         0
total              0
status             0
cidade             0
estado             0
pais               0
cep                0
frete              0
pagamento          0
vendedor        3518
marca              0
Frete_Check2       0
dtype: int64


TRATAMENTO DO VENDEDOR

In [39]:
limp[limp.vendedor.isnull()].head(6)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,Frete_Check2
112,52388,2018-07-30,07:10:51,MIGUEL ANTÔNIO DE DECCO OLIVEIRA,Manteiga,8.77,5,56.85,Pagamento Confirmado,Juiz de Fora,MG,Brasil,36000-000,13.0,Cartão de Crédito,NaN,Aviação,13.0
425,46034,2019-12-28,14:14:40,RAFAEL HENRIQUE COSTA BARNABE MARAZO,Suco de Laranja,825.00,5,4125.00,Aguardando Pagamento,Leopoldina,MG,Brasil,367XXXXX,0.0,Boleto,NaN,Prats,0.0
525,30444,2022-09-11,21:39:42,LUCAS GONÇALVES DONADONI,Café,10.33,2,20.66,Em Separação,Niterói,RJ,Brasil,24000-000,-0.0,Transferência Bancária,NaN,3 Corações,-0.0
543,41142,2020-10-16,12:47:08,GABRIEL CARLOS DE SOUZA,Vinho,28.89,1,28.89,Pagamento Confirmado,Coronel Xavier Chaves,MG,Brasil,36330-000,0.0,Cartão de Crédito,NaN,Miolo,0.0
544,194,2020-06-28,12:28:55,MATEUS Araujo Kuhn,Suco de Laranja,6.86,6,51.16,Em Separação,Lima Duarte,MG,Brasil,36140-000,10.0,Pix,NaN,Prats,10.0
623,55288,2020-05-08,14:08:29,SAMUEL HENRIQUE CAÇADOR,Queijo Mussarela,17.69,1,17.69,Entregue,Niterói,RJ,Brasil,24000-000,0.0,Transferência Bancária,NaN,Quatá,0.0


In [40]:
limp['vendedor'] = limp['vendedor'].fillna('DESCONHECIDO')

In [41]:
limp.head(6)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,Frete_Check2
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,16.87,13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre,20.0
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,3.25,3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini,0.0
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,1.63,9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba,22.0
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,Carvão,8.74,4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás,20.0
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,9.48,2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações,0.0
5,38623,2018-02-19,17:32:01,LUCAS ANTÔNIO DE SOUZA NETO,Café,10.16,1,20.16,Entregue com Sucesso,Resende Costa,MG,Brasil,36340-000,10.0,Transferência Bancária,VICTOR GONÇALVES DONADONI,3 Corações,10.0


TRATANDO AS DUPLICATAS

Removendo as Duplicatas com Base em todas as colunas

In [ ]:
limp = limp.drop_duplicates()

Adicionando Status Correto

In [ ]:
status_correto = {
    'PC': 'Pagamento Confirmado',
    'Pgto Confirmado': 'Pagamento Confirmado',
    'AP': 'Aguardando Pagamento',
    'Aguardando Pgto': 'Aguardando Pagamento',
    'aguardando pagamento': 'Aguardando Pagamento',
    'Sep': 'Em Separação',
    'Separando': 'Em Separação',
    'Transp': 'Em Transporte',
    'Transportando': 'Em Transporte',
    'Entg': 'Entregue com Sucesso',
    'Entregue': 'Entregue com Sucesso'
}
limp['status'] = limp['status'].replace(status_correto)

In [43]:
for col in limp.select_dtypes(include='object').columns:
    limp[col] = limp[col].astype(str).str.strip()

In [45]:
limp = limp.drop_duplicates()

In [48]:
colunas_numericas = ['valor', 'quantidade', 'frete', 'total']
for col in colunas_numericas:
    if col in limp.columns:
        limp = limp[limp[col] >= 0]

In [50]:
if 'produto' in limp.columns:
    limp['produto'] = limp['produto'].astype(str).str.strip().str.lower()

In [51]:
limp = limp[
    (limp["valor"] >= 0) &
    (limp["quantidade"] >= 0) &
    (limp["frete"] >= 0) &
    (limp["total"] >= 0)
]

In [52]:
print(limp.isnull().sum())

id_da_compra    0
data            0
hora            0
cliente         0
produto         0
valor           0
quantidade      0
total           0
status          0
cidade          0
estado          0
pais            0
cep             0
frete           0
pagamento       0
vendedor        0
marca           0
Frete_Check2    0
dtype: int64


In [53]:
print(limp[limp.isnull().any(axis=1)])

Empty DataFrame
Columns: [id_da_compra, data, hora, cliente, produto, valor, quantidade, total, status, cidade, estado, pais, cep, frete, pagamento, vendedor, marca, Frete_Check2]
Index: []


In [ ]:
limp = limp.dropna()

In [ ]:
limp = limp.drop_duplicates(subset=['produto'])

Realozando o Mapeamento de Produtos

In [55]:
mapeamento_produtos = {
    'produto a': 'produto a',
    'produto b': 'produto b'
}

limp['produto'] = limp['produto'].map(mapeamento_produtos).fillna(limp['produto'])

In [57]:
dados_faltantes = limp.isna().sum().sum()

In [59]:
linhas_com_nan = limp[limp.isnull().any(axis=1)]

Lidando com Valores Inconsistentes

In [60]:
valores_inconsistentes = {}
for col in ['valor', 'quantidade', 'frete', 'total']:
    if col in limp.columns:
        inconsistentes = limp[limp[col] <= 0]
        if not inconsistentes.empty:
            valores_inconsistentes[col] = inconsistentes

In [63]:
limp = limp[limp['produto'].str.match('^[A-Za-z0-9\s]+$')] 

In [65]:
valores_inconsistentes = limp[~limp['produto'].str.match('^[A-Za-z0-9\s]+$')].shape[0]

In [ ]:
limp['produto'] = limp['produto'].replace('Produto@123', 'ProdutoCorreto')limp['produto'] = limp['produto'].replace('Produto@123', 'ProdutoCorreto')

In [66]:
produtos_unicos = limp['produto'].unique()

In [67]:
produtos_unicos = limp['produto'].astype(str).unique()
produtos_duplicados = limp['produto'].duplicated().sum()  
padronizacao_produtos = limp['produto'].str.lower().unique()

In [68]:
import unicodedata
limp['produto'] = limp['produto'].apply(lambda x: ''.join(c for c in unicodedata.normalize('NFD', x) if unicodedata.category(c) != 'Mn'))

In [70]:
produtos_unicos = limp['produto'].unique()

In [72]:
dados_faltantes = limp.isna().sum()  
total_faltante = limp.isna().sum().sum()

In [73]:
print(limp['produto'].unique())

['queijo mussarela' 'molho de tomate' 'condicionador' 'sabonete'
 'manteiga' 'vinho' 'shampoo' 'presunto' 'cerveja' 'papel toalha'
 'pasta de dente' 'amaciante' 'desinfetante' 'refrigerante' 'arroz'
 'farinha de trigo' 'biscoito recheado' 'sal' 'suco de laranja'
 'leite integral' 'detergente' 'scl' 'sucoyde laranja' 'farinha de tripo'
 'tal' 'caff' 'biscoitq recheado']


In [75]:
limp['produto'] = limp['produto'].astype(str).str.strip().str.lower()

In [77]:
limp = limp[limp['cep'].astype(str).str.match(r'^\d{5}-\d{3}$')]

In [78]:
for col in limp.select_dtypes(include='object').columns:
    limp[col] = limp[col].str.strip()

In [79]:
limp.to_csv('final_corrigido.csv', index=False, encoding='utf-8-sig')

CORREÇÃO DOS ESTADOS

In [80]:
estado_correto = {
    'São Paulo': 'SP',
    'MTSa': 'MT',
    'PSC': 'SC'
}
limp['estado'] = limp['estado'].replace(estado_correto)

 TRATAMENTO DE CEPs

In [81]:
limp = limp[limp['cep'].astype(str).str.match(r'^\d{5}-\d{3}$')]

REMOÇÃO DE ESPAÇOS EXTRAS EM TODAS AS COLUNAS

In [82]:
for col in limp.select_dtypes(include='object').columns:
    limp[col] = limp[col].str.strip()

In [88]:
limp.to_csv('final_corrigido.csv', index=False, encoding='utf-8-sig')

1. Padronização de Datas e Horários

Datas: Todos os registros foram uniformizados para o formato AAAA-MM-DD, seguindo o padrão internacional, o que facilita a ordenação e análise temporal.

Horários: Os registros de tempo foram ajustados para o sistema de 24 horas (HH:MM:SS), eliminando ambiguidades e garantindo consistência em análises que envolvem períodos.

2. Normalização de Valores Numéricos:

   
Campos Monetários e Quantitativos: Colunas como valor, quantidade, frete e total foram convertidas para os tipos float ou int, conforme adequação.

Remoção de Símbolos e Ajuste de Formato: Caracteres como "R$" e vírgulas (,) como separadores decimais foram substituídos por pontos (.), utilizando expressões regulares (biblioteca re) para garantir precisão na conversão.

3. Tratamento de Dados Inconsistentes
Campos Vazios:

Registros com informações críticas ausentes (ex.: valor, quantidade) foram excluídos para manter a integridade da base.

Dados não essenciais com valores faltantes foram preenchidos estrategicamente:

Frete ausente: Substituído pela média dos valores existentes.

Datas faltantes: Preenchidas com a moda (valor mais frequente).

Registros Duplicados:

Foram identificados e removidos X registros duplicados, priorizando as entradas mais completas ou mais recentes.

4. Validação e Correção do Campo "Total"

Para garantir a precisão dos cálculos, o campo total foi recalculado seguindo a fórmula:

    total = (valor * quantidade) + frete

Quando foram encontradas divergências, os valores foram reajustados automaticamente com base nessa regra, assegurando consistência nos dados.